In [1]:
import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

dataFileColumns = ["timestamp", "stage", "endTimestamp"]
outputFileName = "fitbitSleepStages.parquet.gzip"

#get list of files in current directroy
files = [f for f in os.listdir("./") if os.path.isfile(f)]

#if the output file isn't already made, make it
if not (outputFileName in files):
    print("making new df")
    emptydf = pd.DataFrame(columns = dataFileColumns)
    emptydf.set_index('timestamp',inplace=True)
    print(emptydf)
        
    #save as a parquet file
    emptydf.to_parquet(outputFileName, compression='gzip') 


df = pd.read_parquet(outputFileName)

print(df)


1.2.4
making new df
Empty DataFrame
Columns: [stage, endTimestamp]
Index: []
Empty DataFrame
Columns: [stage, endTimestamp]
Index: []


In [2]:
def fitbitSleepStageToStandard(stageName):
    if stageName == "restless":
        return 2
    if stageName == "asleep":
        return 3
    if stageName == "awake":
        return 0
    if stageName == "wake":
        return 0
    if stageName == "light":
        return 2
    if stageName == "deep":
        return 3
    if stageName == "rem":
        return 4
    print(stageName)


In [3]:
# there's a list of sleep objects for each sleep session
# we want start time and assign the phoenix timezone
#then there's a data list of sleep stage json objects
def addJSONdataTodf(jd, returnDF):
    for sleepSession in jd:
        #print(sleepSession['levels']['data'])
        #print(sleepSession)
        
        for sleepStage in sleepSession['levels']['data']:
            toadd = []
            stageTime = AZtz.localize(pd.to_datetime(sleepStage['dateTime'], format='%Y-%m-%dT%H:%M:%S.%f')).astimezone(utc)
            #toadd.append(stageTime)
            toadd.append(fitbitSleepStageToStandard(sleepStage['level']))
            toadd.append(stageTime + datetime.timedelta(seconds=int(sleepStage['seconds'])))
            returnDF.loc[stageTime] = toadd
    return returnDF


In [4]:
datDir = "./fitbit/"

sleepFiles = os.listdir(datDir)

print(sleepFiles)
for sf in sleepFiles:
  with open(datDir + sf) as f:
    jsonData = json.load(f)
  df = addJSONdataTodf(jsonData,df)
  print("imported: " + sf)

['sleep-2020-06-11.json', 'sleep-2020-05-12.json', 'sleep-2021-04-07.json', 'sleep-2021-02-06.json', 'sleep-2020-08-10.json', 'sleep-2020-11-08.json', 'sleep-2021-07-06.json', 'sleep-2021-05-07.json', 'sleep-2020-09-09.json', 'sleep-2020-10-09.json', 'sleep-2020-12-08.json', 'sleep-2021-01-07.json', 'sleep-2021-03-08.json', 'sleep-2021-08-05.json', 'sleep-2020-07-11.json', 'sleep-2021-06-06.json']
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake
wake

In [5]:
df.sort_index(inplace=True)
print(df)

                          stage              endTimestamp
timestamp                                                
2020-05-13 20:43:00+00:00     2 2020-05-13 20:45:00+00:00
2020-05-13 20:45:00+00:00     3 2020-05-13 21:41:00+00:00
2020-05-13 21:41:00+00:00     2 2020-05-13 21:42:00+00:00
2020-05-13 21:42:00+00:00     3 2020-05-13 21:43:00+00:00
2020-05-14 08:21:00+00:00   NaT 2020-05-14 08:32:30+00:00
...                         ...                       ...
2021-08-09 13:39:30+00:00     4 2021-08-09 14:07:00+00:00
2021-08-09 14:07:00+00:00     2 2021-08-09 14:47:00+00:00
2021-08-09 14:47:00+00:00     3 2021-08-09 14:56:30+00:00
2021-08-09 14:56:30+00:00   NaT 2021-08-09 15:01:30+00:00
2021-08-09 15:01:30+00:00     2 2021-08-09 15:44:30+00:00

[14033 rows x 2 columns]


In [6]:
df.sort_index(inplace=True)
df.to_parquet(outputFileName, compression='gzip') 